This Colab uses Meterstick to reproduce some results of Card's minimum wage [study](https://davidcard.berkeley.edu/papers/njmin-aer.pdf).

In [ ]:
# !pip install meterstick

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.1/101.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/google/meterstick.git
import sys, os
sys.path.append(os.getcwd())

Cloning into 'meterstick'...
remote: Enumerating objects: 1068, done.
remote: Counting objects: 100% (365/365), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 1068 (delta 251), reused 225 (delta 223), pack-reused 703 (from 2)
Receiving objects: 100% (1068/1068), 1.85 MiB | 4.89 MiB/s, done.
Resolving deltas: 100% (738/738), done.


In [ ]:
import numpy as np
import pandas as pd
from io import StringIO
from meterstick import *
from meterstick.models import *

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/google/meterstick/refs/heads/master/publications/a_grammar_of_data_analysis/datasets/minimum_wage.csv', sep=';')
df['STATE_NAME'] = df.STATE.replace([0, 1], ['PA', 'NJ'])
df.CHAIN = df.CHAIN.replace(
    [1, 2, 3, 4], ['Burger King', 'KFC', 'Roy Rogers', "Wendy's"]
)
df['EMPTOT'] = df.EMPPT * 0.5 + df.EMPFT + df.NMGRS
df['EMPTOT2'] = df.EMPPT2 * 0.5 + df.EMPFT2 + df.NMGRS2
df['DEMP'] = df.EMPTOT2 - df.EMPTOT
df['PCHEMPC'] = 2 * (df.EMPTOT2 - df.EMPTOT) / (df.EMPTOT2 + df.EMPTOT)
df['DWAGE'] = df.WAGE_ST2 - df.WAGE_ST
df['CLOSED'] = df.STATUS2 == 3
# df['GAP'] = np.where((df.STATE_NAME == 'PA') | (df.WAGE_ST >= 5.05), 0, (5.05 - df.WAGE_ST) / df.WAGE_ST)
df.head()

,SHEET,CHAIN,CO_OWNED,STATE,SOUTHJ,CENTRALJ,NORTHJ,PA1,PA2,SHORE,...,PENTREE2,NREGS2,NREGS112,STATE_NAME,EMPTOT,EMPTOT2,DEMP,PCHEMPC,DWAGE,CLOSED
0,46,Burger King,0,0,0,0,0,1,0,0,...,0.94,4.0,4,PA,40.50,24.0,-16.50,-0.511628,NaN,False
1,49,KFC,0,0,0,0,0,1,0,0,...,2.35,4.0,4,PA,13.75,11.5,-2.25,-0.178218,NaN,False
2,506,KFC,1,0,0,0,0,1,0,0,...,2.33,4.0,3,PA,8.50,10.5,2.00,0.210526,NaN,False
3,56,Wendy's,1,0,0,0,0,1,0,0,...,0.87,2.0,2,PA,34.00,20.0,-14.00,-0.518519,0.25,False
4,61,Wendy's,1,0,0,0,0,1,0,0,...,0.95,2.0,2,PA,24.00,35.5,11.50,0.386555,-0.75,False


# Descriptive Stats

Codes below reproduce the distribution of store types in Table 2 of the paper.

In [ ]:
Distribution('CHAIN', Count('SHEET', name='Store Types')).compute_on(df, 'STATE_NAME')

Distribution of Store Types
STATE_NAME CHAIN                                   
NJ         Burger King                     0.410876
           KFC                             0.205438
           Roy Rogers                      0.247734
           Wendy's                         0.135952
PA         Burger King                     0.443038
           KFC                             0.151899
           Roy Rogers                      0.215190
           Wendy's                         0.189873

# Difference in Differences

Codes below reproduce some numbers in Table 3 of the paper, including
- mean employment rates for NJ and PA before and after the minimum wage increase,
- differences between NJ and PA for each period,
- and the difference-in-differences.

Using Jackknife mostly gives an exact match on standard errors and using Bootstrap gives close numbers too.

In [ ]:
np.random.seed(0)
fte_before = Mean('EMPTOT', name='FTE employment before')
fte_after = Mean('EMPTOT2', name='FTE employment after')
(MetricList((fte_before, fte_after))
| Jackknife('SHEET')  # SHEET is almost unique for each row so we basically Jackknife by rows.
| compute_on(df, 'STATE_NAME'))

Metric     FTE employment before              FTE employment after  \
                           Value Jackknife SE                Value   
STATE_NAME                                                           
NJ                     20.439408     0.508285            21.027429   
PA                     23.331169     1.351374            21.165584   

Metric                   
           Jackknife SE  
STATE_NAME               
NJ             0.520339  
PA             0.943378

In [ ]:
np.random.seed(0)
(MetricList((fte_before, fte_after))
| Bootstrap()
| compute_on(df, 'STATE_NAME'))

Metric     FTE employment before              FTE employment after  \
                           Value Bootstrap SE                Value   
STATE_NAME                                                           
NJ                     20.439408     0.506039            21.027429   
PA                     23.331169     1.351847            21.165584   

Metric                   
           Bootstrap SE  
STATE_NAME               
NJ             0.515040  
PA             0.950651

In [ ]:
np.random.seed(0)
fte_diff_before = AbsoluteChange('STATE_NAME', 'PA', fte_before)
fte_diff_after = AbsoluteChange('STATE_NAME', 'PA', fte_after)
did = (fte_diff_after - fte_diff_before).set_name('FTE DID')
all_diffs = MetricList((fte_diff_before, fte_diff_after, did))
(all_diffs
| Jackknife('SHEET')
| compute_on(df))

Metric     FTE employment before Absolute Change               \
                                           Value Jackknife SE   
STATE_NAME                                                      
NJ                                     -2.891761     1.447435   

Metric     FTE employment after Absolute Change                FTE DID  \
                                          Value Jackknife SE     Value   
STATE_NAME                                                               
NJ                                    -0.138155     1.079166  2.753606   

Metric                   
           Jackknife SE  
STATE_NAME               
NJ             1.316239

In [ ]:
np.random.seed(0)
(all_diffs
| Bootstrap()
| compute_on(df))

Metric     FTE employment before Absolute Change               \
                                           Value Bootstrap SE   
STATE_NAME                                                      
NJ                                     -2.891761     1.438856   

Metric     FTE employment after Absolute Change                FTE DID  \
                                          Value Bootstrap SE     Value   
STATE_NAME                                                               
NJ                                    -0.138155     1.065946  2.753606   

Metric                   
           Bootstrap SE  
STATE_NAME               
NJ             1.315439

If the data is in long format, DID can be expressed as chained `AbsoluteChange`.

In [ ]:
df_long = pd.melt(df, ['SHEET', 'STATE_NAME'], ['EMPTOT', 'EMPTOT2'], 'WAVE', 'EMP')
df_long.head()

,SHEET,STATE_NAME,WAVE,EMP
0,46,PA,EMPTOT,40.50
1,49,PA,EMPTOT,13.75
2,506,PA,EMPTOT,8.50
3,56,PA,EMPTOT,34.00
4,61,PA,EMPTOT,24.00


In [ ]:
(Mean('EMP')
| AbsoluteChange('STATE_NAME', 'PA')
| AbsoluteChange('WAVE', 'EMPTOT')
| Jackknife('SHEET')
| compute_on(df_long))

Metric             mean(EMP) Absolute Change Absolute Change             
                                                       Value Jackknife SE
WAVE    STATE_NAME                                                       
EMPTOT2 NJ                                          2.753606     1.316239

In [ ]:
np.random.seed(0)
(Mean('EMP')
| AbsoluteChange('STATE_NAME', 'PA')
| AbsoluteChange('WAVE', 'EMPTOT')
# Bootstrap by SHEET makes sure the EMP before and after for one store are always sampled together.
| Bootstrap('SHEET')
| compute_on(df_long))

Metric             mean(EMP) Absolute Change Absolute Change             
                                                       Value Bootstrap SE
WAVE    STATE_NAME                                                       
EMPTOT2 NJ                                          2.753606     1.321578

Alternatively, the Difference-in-Differences (DID) estimate can be directly obtained through linear regression by fitting the model:  
`EMP = β0 + β1 * IS_NJ + β2 * IS_WAVE2 + β3 * IS_NJ_AND_WAVE2 + ε`,  
Where:
- `EMP` represents the employment rate.
- `IS_NJ` is an indicator variable equal to 1 for observations in New Jersey and 0 for Pennsylvania.
- `IS_WAVE2` is an indicator variable equal to 1 for observations in the period after the minimum wage increase (Wave 2) and 0 for the period before (Wave 1).
- `IS_NJ_AND_WAVE2` is an interaction term, equal to 1 only for observations in New Jersey after the minimum wage increase, and 0 otherwise.
- `ϵ` is the error term.

In this specification, the coefficients hold specific meanings relevant to DID. Specifically,
- β1 will quantify the initial difference in employment rates between New Jersey and Pennsylvania before the minimum wage change (the `FTE employment before Absolute Change` above).
- β2 will capture the employment rate change within the control group (Pennsylvania) across the two time periods.
- β3 will precisely match the calculated DID estimate.

In [ ]:
df_for_did_reg = pd.get_dummies(df_long[df_long.EMP.notna()], columns=['STATE_NAME', 'WAVE'])
df_for_did_reg['IS_NJ_AND_WAVE2'] = df_for_did_reg.STATE_NAME_NJ * df_for_did_reg.WAVE_EMPTOT2
df_for_did_reg['ROW_ID'] = range(len(df_for_did_reg))
# Fit 'EMP ~ IS_NJ + IS_WAVE2 + IS_NJ * IS_WAVE2' using each row as a data point.
# Because each row is a data point, Mean is a no-op. It can be replaced with Sum/Min/Max and so on.
dd = LinearRegression(
    Mean('EMP'),
    (Mean('STATE_NAME_NJ'), Mean('WAVE_EMPTOT2'), Mean('IS_NJ_AND_WAVE2')),
    'ROW_ID',
    name='DID regression'
)
dd | Jackknife('SHEET') | compute_on(df_for_did_reg, melted=True)

,Value,Jackknife SE
Metric,,
DID regression Coefficient: intercept,23.331169,1.358345
DID regression Coefficient: mean(STATE_NAME_NJ),-2.891761,1.447435
DID regression Coefficient: mean(WAVE_EMPTOT2),-2.165584,1.228707
DID regression Coefficient: mean(IS_NJ_AND_WAVE2),2.753606,1.316239


# Regressions

Codes below reproduce the first two models in Table 4 of the paper. The `Coefficient: mean(STATE)` correspond to the coefficient of 'New Jersey dummy` in the paper. See the "check.sas" file in the original [dataset](https://davidcard.berkeley.edu/data_sets.html) for model specifications.

In [ ]:
df_with_dummies = pd.get_dummies(df, columns=['CHAIN'])
np.random.seed(0)
# SUBSET OF STORES WITH VALID WAGES 2 WAVES (OR CLOSED W-2)
where = ('DEMP.notna()', '(CLOSED == 1) | (CLOSED == 0 & DWAGE.notna())')
m1 = LinearRegression(
    Mean('DEMP'), Mean('STATE'), 'SHEET', name='Model 1', where=where
)
m2 = LinearRegression(
    Mean('DEMP'),
    [
        Mean('STATE'),
        Mean('CHAIN_Burger King'),
        Mean('CHAIN_KFC'),
        Mean('CHAIN_Roy Rogers'),
        Mean('CO_OWNED'),
    ],
    'SHEET',
    where=where,
    name='Model 2',
)
models = MetricList((m1, m2))
models | Bootstrap() | compute_on(df_with_dummies, melted=True)

,Value,Bootstrap SE
Metric,,
Model 1 Coefficient: intercept,-2.142245,1.073610
Model 1 Coefficient: mean(STATE),2.337095,1.131986
Model 2 Coefficient: intercept,-2.232424,1.445775
Model 2 Coefficient: mean(STATE),2.299346,1.140145
Model 2 Coefficient: mean(CHAIN_Burger King),0.541523,1.293879
Model 2 Coefficient: mean(CHAIN_KFC),1.037672,1.255976
Model 2 Coefficient: mean(CHAIN_Roy Rogers),-1.665830,1.288243
Model 2 Coefficient: mean(CO_OWNED),0.290020,0.712711
